## Dataset Overview

The dataset contains sensor measurements collected from a manufacturing facility over time. It includes environmental readings from multiple zones within the factory, external weather conditions, and energy consumption metrics. The primary target variable is `equipment_energy_consumption`, which represents the energy used by manufacturing equipment in Wh (watt-hours).

## Feature Descriptions

### Time Information
- `timestamp`: Date and time of the measurement (format: YYYY-MM-DD HH:MM:SS)

### Energy Metrics
- `equipment_energy_consumption`: Energy consumption of manufacturing equipment in Wh (target variable)
- `lighting_energy`: Energy consumption of lighting systems in Wh

### Zone Measurements
The factory is divided into 9 distinct zones, each equipped with temperature and humidity sensors:

#### Zone 1 - Main Production Area
- `zone1_temperature`: Temperature in Zone 1 (°C)
- `zone1_humidity`: Relative humidity in Zone 1 (%)

#### Zone 2 - Assembly Line
- `zone2_temperature`: Temperature in Zone 2 (°C)
- `zone2_humidity`: Relative humidity in Zone 2 (%)

#### Zone 3 - Quality Control
- `zone3_temperature`: Temperature in Zone 3 (°C)
- `zone3_humidity`: Relative humidity in Zone 3 (%)

#### Zone 4 - Packaging Area
- `zone4_temperature`: Temperature in Zone 4 (°C)
- `zone4_humidity`: Relative humidity in Zone 4 (%)

#### Zone 5 - Raw Material Storage
- `zone5_temperature`: Temperature in Zone 5 (°C)
- `zone5_humidity`: Relative humidity in Zone 5 (%)

#### Zone 6 - Loading Bay
- `zone6_temperature`: Temperature in Zone 6 (°C)
- `zone6_humidity`: Relative humidity in Zone 6 (%)

#### Zone 7 - Office Space
- `zone7_temperature`: Temperature in Zone 7 (°C)
- `zone7_humidity`: Relative humidity in Zone 7 (%)

#### Zone 8 - Control Room
- `zone8_temperature`: Temperature in Zone 8 (°C)
- `zone8_humidity`: Relative humidity in Zone 8 (%)

#### Zone 9 - Staff Area
- `zone9_temperature`: Temperature in Zone 9 (°C)
- `zone9_humidity`: Relative humidity in Zone 9 (%)

### External Weather Conditions
- `outdoor_temperature`: Outside temperature (°C)
- `outdoor_humidity`: Outside relative humidity (%)
- `atmospheric_pressure`: Atmospheric pressure (mm Hg)
- `wind_speed`: Wind speed (m/s)
- `visibility_index`: Visibility measure (km)
- `dew_point`: Dew point temperature (°C)

### Additional Variables
- `random_variable1`: Random variable generated for the dataset
- `random_variable2`: Random variable generated for the dataset

## Target Variable Details

The main target variable is `equipment_energy_consumption`, which represents the energy consumption of manufacturing equipment measured in watt-hours (Wh). This is a continuous variable that typically ranges from 10 to 1080 Wh in the dataset. The distribution is right-skewed, with most values concentrated in the lower range.

In [64]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import calendar

import statsmodels.api as sm
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression,Ridge,Lasso
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from xgboost import XGBRegressor
from sklearn.preprocessing import PolynomialFeatures
from sklearn.decomposition import PCA

import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense,LeakyReLU,BatchNormalization,Dropout,ReLU
from tensorflow.keras.utils import plot_model
from tensorflow.keras.optimizers import Adam,RMSprop
from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow.keras.losses import Huber
from tensorflow.keras.callbacks import EarlyStopping,ModelCheckpoint,TensorBoard
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.models import load_model
from tensorflow.keras.metrics import MeanSquaredError, MeanAbsoluteError

import warnings
warnings.filterwarnings('ignore')

In [65]:
## load data

df = pd.read_csv("/Users/phoenix/Documents/GitHub/DS-Intern-Assignment-Vaibhav_Pandey/data/data.csv")
df.head()

,timestamp,equipment_energy_consumption,lighting_energy,zone1_temperature,zone1_humidity,zone2_temperature,zone2_humidity,zone3_temperature,zone3_humidity,zone4_temperature,...,zone9_temperature,zone9_humidity,outdoor_temperature,atmospheric_pressure,outdoor_humidity,wind_speed,visibility_index,dew_point,random_variable1,random_variable2
0,2016-01-11 17:00:00,60.0,-77.78778596503064,33.74660933896648,47.59666666666671,19.2,44.790000,19.79,NaN,19.000000,...,17.033333,45.53,6.600000,733.5,92.0,7.000000,63.000000,5.300000,13.275433,13.275433
1,2016-01-11 17:10:00,60.0,30.0,19.89,46.69333333333329,19.2,44.722500,19.79,44.790000,19.000000,...,17.066667,45.56,6.483333,733.6,92.0,6.666667,59.166667,5.200000,18.606195,18.606195
2,2016-01-11 17:20:00,50.0,30.0,19.89,46.3,19.2,44.626667,19.79,44.933333,35.921144,...,17.000000,45.50,6.366667,733.7,92.0,6.333333,55.333333,5.100000,28.642668,28.642668
3,2016-01-11 17:30:00,50.0,40.0,33.74660933896648,46.0666666666667,19.2,44.590000,19.79,45.000000,NaN,...,17.000000,45.40,6.250000,733.8,92.0,6.000000,51.500000,37.673716,45.410389,45.410389
4,2016-01-11 17:40:00,60.0,40.0,19.89,46.33333333333329,19.2,44.530000,19.79,45.000000,18.890000,...,4.476511,45.40,6.133333,733.9,92.0,5.666667,47.666667,4.900000,10.084097,10.084097


#### Data Preprocessing

In [66]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16857 entries, 0 to 16856
Data columns (total 29 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   timestamp                     16857 non-null  object 
 1   equipment_energy_consumption  16013 non-null  object 
 2   lighting_energy               16048 non-null  object 
 3   zone1_temperature             15990 non-null  object 
 4   zone1_humidity                16056 non-null  object 
 5   zone2_temperature             16004 non-null  object 
 6   zone2_humidity                15990 non-null  float64
 7   zone3_temperature             16055 non-null  float64
 8   zone3_humidity                15979 non-null  float64
 9   zone4_temperature             16041 non-null  float64
 10  zone4_humidity                16076 non-null  float64
 11  zone5_temperature             16019 non-null  float64
 12  zone5_humidity                16056 non-null  float64
 13  z

In [67]:
df.describe()

,zone2_humidity,zone3_temperature,zone3_humidity,zone4_temperature,zone4_humidity,zone5_temperature,zone5_humidity,zone6_temperature,zone6_humidity,zone7_temperature,...,zone9_temperature,zone9_humidity,outdoor_temperature,atmospheric_pressure,outdoor_humidity,wind_speed,visibility_index,dew_point,random_variable1,random_variable2
count,15990.000000,16055.000000,15979.000000,16041.000000,16076.000000,16019.000000,16056.000000,16009.000000,16010.000000,16063.000000,...,16084.000000,15969.000000,16051.000000,16015.000000,16058.000000,16029.000000,16042.000000,16031.000000,16031.000000,16033.000000
mean,39.494553,21.665733,38.201314,20.239922,37.945608,19.052613,50.289131,6.469934,59.162913,19.672472,...,18.851085,40.318199,6.218582,755.757602,78.977595,4.195988,38.457150,2.783507,24.854981,25.093780
std,10.129513,2.594309,10.144388,2.783050,10.769813,2.346158,18.722516,8.867993,52.657787,2.879815,...,2.528783,11.168678,7.554684,13.643585,28.565660,4.409729,21.318655,6.094732,26.214960,25.523631
min,-77.265503,6.543921,-71.406273,4.613485,-81.446225,5.921094,-141.640143,-42.987365,-353.393026,3.578021,...,4.476511,-81.582494,-37.524768,678.159940,-221.668765,-20.929527,-82.329792,-32.098095,-120.170177,-120.409590
25%,37.757500,20.533333,36.592500,19.266667,35.200000,18.061111,45.290000,2.930000,37.066667,18.500000,...,17.890000,38.230000,3.000000,750.800000,71.000000,2.000000,29.000000,0.450000,12.179924,12.193789
50%,40.293333,21.767500,38.400000,20.290000,38.090000,19.050000,48.854429,6.263333,62.766667,19.600000,...,18.890000,40.363333,6.000000,756.200000,84.166667,4.000000,40.000000,2.750000,24.866978,24.834261
75%,43.000000,22.760000,41.433333,21.356667,41.560833,20.100000,53.918333,9.690000,86.590000,21.000000,...,20.200000,43.790000,9.250000,762.100000,91.988095,6.000000,40.000000,5.308333,37.950236,37.971764
max,77.265503,36.823982,71.406273,35.921144,81.446225,32.157594,141.640143,55.932271,353.393026,35.712952,...,33.238012,81.582494,49.853906,833.392675,221.668765,29.318719,159.606156,37.673716,170.156325,170.329617


In [68]:
df.isnull().sum()

timestamp                         0
equipment_energy_consumption    844
lighting_energy                 809
zone1_temperature               867
zone1_humidity                  801
zone2_temperature               853
zone2_humidity                  867
zone3_temperature               802
zone3_humidity                  878
zone4_temperature               816
zone4_humidity                  781
zone5_temperature               838
zone5_humidity                  801
zone6_temperature               848
zone6_humidity                  847
zone7_temperature               794
zone7_humidity                  805
zone8_temperature               848
zone8_humidity                  777
zone9_temperature               773
zone9_humidity                  888
outdoor_temperature             806
atmospheric_pressure            842
outdoor_humidity                799
wind_speed                      828
visibility_index                815
dew_point                       826
random_variable1            

In [69]:
df.dtypes

timestamp                        object
equipment_energy_consumption     object
lighting_energy                  object
zone1_temperature                object
zone1_humidity                   object
zone2_temperature                object
zone2_humidity                  float64
zone3_temperature               float64
zone3_humidity                  float64
zone4_temperature               float64
zone4_humidity                  float64
zone5_temperature               float64
zone5_humidity                  float64
zone6_temperature               float64
zone6_humidity                  float64
zone7_temperature               float64
zone7_humidity                  float64
zone8_temperature               float64
zone8_humidity                  float64
zone9_temperature               float64
zone9_humidity                  float64
outdoor_temperature             float64
atmospheric_pressure            float64
outdoor_humidity                float64
wind_speed                      float64


In [70]:
# Convert specified columns to float

for col in ['equipment_energy_consumption', 'lighting_energy', 'zone1_temperature', 'zone1_humidity', 'zone2_temperature']:
    df[col] = pd.to_numeric(df[col], errors='coerce')

In [71]:
df.dtypes

timestamp                        object
equipment_energy_consumption    float64
lighting_energy                 float64
zone1_temperature               float64
zone1_humidity                  float64
zone2_temperature               float64
zone2_humidity                  float64
zone3_temperature               float64
zone3_humidity                  float64
zone4_temperature               float64
zone4_humidity                  float64
zone5_temperature               float64
zone5_humidity                  float64
zone6_temperature               float64
zone6_humidity                  float64
zone7_temperature               float64
zone7_humidity                  float64
zone8_temperature               float64
zone8_humidity                  float64
zone9_temperature               float64
zone9_humidity                  float64
outdoor_temperature             float64
atmospheric_pressure            float64
outdoor_humidity                float64
wind_speed                      float64


In [72]:
## Dropping rows with negative values in 'equipment_energy_consumption' and 'lighting_energy'

df = df[~(df['lighting_energy'] < 0)]
df = df[~(df['equipment_energy_consumption'] < 0)]

#### I noticed that there were some negative entries in 'equipment_energy_consumption' and 'lighting_energy', since these are units of power in Wh. It implies that there is some error in capturing the correct reading from the sensor hence i decided to remove them completelyy from the dataset.

In [73]:
df.shape

(16318, 29)

In [74]:
humidity_columns = [col for col in df.columns if 'humidity' in col.lower() and col != 'outdoor_humidity']

# Combine all humidity columns (including outdoor_humidity)
all_humidity_columns = humidity_columns + ['outdoor_humidity']

invalid_humidity_mask = df[all_humidity_columns].lt(0).any(axis=1)

affected_rows_count = invalid_humidity_mask.sum()

In [75]:
print(f"Affected rows: {affected_rows_count}")
print(f"Percentage of dataset affected: {affected_rows_count / len(df) * 100:.2f}%")

Affected rows: 2337
Percentage of dataset affected: 14.32%


In [76]:
def drop_negative_humidity(df):
    """
    Function to drop rows with negative humidity values.
    Action:
    Drop rows with negative humidity values in any humidity-related column.
    """
    
    humidity_cols = [col for col in df.columns if "humidity" in col.lower()]
    mask = (df[humidity_cols] < 0).any(axis=1)
    print(f"Dropping {mask.sum()} rows with negative humidity values.")

    df_cleaned = df[~mask].reset_index(drop=True)
    return df_cleaned

In [77]:
df = drop_negative_humidity(df)
df.shape

Dropping 2337 rows with negative humidity values.


(13981, 29)

#### I noticed that there were some negative entries in the humidity-related columns, including 'zone1_humidity' to 'zone9_humidity' and 'outdoor_humidity'. Since humidity is a percentage and cannot logically take negative values, this implies an error in capturing the correct readings from the sensors. Upon analysis, I found that 2,337 rows (14.32% of the dataset) had at least one invalid humidity value. To ensure data quality and avoid introducing bias through imputation, I decided to remove these rows entirely from the dataset.

In [78]:
df.isnull().sum()

timestamp                         0
equipment_energy_consumption    779
lighting_energy                 722
zone1_temperature               782
zone1_humidity                  733
zone2_temperature               749
zone2_humidity                  733
zone3_temperature               678
zone3_humidity                  754
zone4_temperature               667
zone4_humidity                  661
zone5_temperature               699
zone5_humidity                  690
zone6_temperature               714
zone6_humidity                  723
zone7_temperature               650
zone7_humidity                  689
zone8_temperature               712
zone8_humidity                  659
zone9_temperature               639
zone9_humidity                  733
outdoor_temperature             681
atmospheric_pressure            696
outdoor_humidity                685
wind_speed                      712
visibility_index                671
dew_point                       689
random_variable1            

In [79]:
## Convert the 'timestamp' column to datetime format

df['timestamp'] = pd.to_datetime(df['timestamp'])

print(df.dtypes)
df.head()

timestamp                       datetime64[ns]
equipment_energy_consumption           float64
lighting_energy                        float64
zone1_temperature                      float64
zone1_humidity                         float64
zone2_temperature                      float64
zone2_humidity                         float64
zone3_temperature                      float64
zone3_humidity                         float64
zone4_temperature                      float64
zone4_humidity                         float64
zone5_temperature                      float64
zone5_humidity                         float64
zone6_temperature                      float64
zone6_humidity                         float64
zone7_temperature                      float64
zone7_humidity                         float64
zone8_temperature                      float64
zone8_humidity                         float64
zone9_temperature                      float64
zone9_humidity                         float64
outdoor_tempe

,timestamp,equipment_energy_consumption,lighting_energy,zone1_temperature,zone1_humidity,zone2_temperature,zone2_humidity,zone3_temperature,zone3_humidity,zone4_temperature,...,zone9_temperature,zone9_humidity,outdoor_temperature,atmospheric_pressure,outdoor_humidity,wind_speed,visibility_index,dew_point,random_variable1,random_variable2
0,2016-01-11 17:10:00,60.0,30.0,19.890000,46.693333,19.2,44.722500,19.79,44.790000,19.000000,...,17.066667,45.56,6.483333,733.6,92.0,6.666667,59.166667,5.200000,18.606195,18.606195
1,2016-01-11 17:20:00,50.0,30.0,19.890000,46.300000,19.2,44.626667,19.79,44.933333,35.921144,...,17.000000,45.50,6.366667,733.7,92.0,6.333333,55.333333,5.100000,28.642668,28.642668
2,2016-01-11 17:30:00,50.0,40.0,33.746609,46.066667,19.2,44.590000,19.79,45.000000,NaN,...,17.000000,45.40,6.250000,733.8,92.0,6.000000,51.500000,37.673716,45.410389,45.410389
3,2016-01-11 17:40:00,60.0,40.0,19.890000,46.333333,19.2,44.530000,19.79,45.000000,18.890000,...,4.476511,45.40,6.133333,733.9,92.0,5.666667,47.666667,4.900000,10.084097,10.084097
4,2016-01-11 17:50:00,50.0,40.0,19.890000,46.026667,19.2,44.500000,19.79,44.933333,18.890000,...,17.000000,NaN,6.016667,734.0,92.0,5.333333,43.833333,4.800000,44.919484,NaN


In [80]:
#  Drop rows with missing target

df = df.dropna(subset=["equipment_energy_consumption"])

In [81]:
df.isnull().sum()

timestamp                         0
equipment_energy_consumption      0
lighting_energy                 670
zone1_temperature               736
zone1_humidity                  693
zone2_temperature               711
zone2_humidity                  694
zone3_temperature               636
zone3_humidity                  699
zone4_temperature               627
zone4_humidity                  623
zone5_temperature               652
zone5_humidity                  643
zone6_temperature               671
zone6_humidity                  681
zone7_temperature               601
zone7_humidity                  652
zone8_temperature               673
zone8_humidity                  626
zone9_temperature               599
zone9_humidity                  691
outdoor_temperature             647
atmospheric_pressure            661
outdoor_humidity                645
wind_speed                      670
visibility_index                631
dew_point                       637
random_variable1            

### Handling Missing Values

In [82]:
# Impute missing values

for col in df.columns:
    if col != 'equipment_energy_consumption' and df[col].isnull().any():
        if df[col].dtype in ['int64', 'float64']:
            df[col].fillna(df[col].median(), inplace=True)

In [83]:
df.isnull().sum()

timestamp                       0
equipment_energy_consumption    0
lighting_energy                 0
zone1_temperature               0
zone1_humidity                  0
zone2_temperature               0
zone2_humidity                  0
zone3_temperature               0
zone3_humidity                  0
zone4_temperature               0
zone4_humidity                  0
zone5_temperature               0
zone5_humidity                  0
zone6_temperature               0
zone6_humidity                  0
zone7_temperature               0
zone7_humidity                  0
zone8_temperature               0
zone8_humidity                  0
zone9_temperature               0
zone9_humidity                  0
outdoor_temperature             0
atmospheric_pressure            0
outdoor_humidity                0
wind_speed                      0
visibility_index                0
dew_point                       0
random_variable1                0
random_variable2                0
dtype: int64

### Feature Engineering

In [84]:
# Extracting date and time features from the timestamp

df['month'] = df['timestamp'].dt.month
df['day'] = df['timestamp'].dt.day
df['dayofweek'] = df['timestamp'].dt.dayofweek
df['hour'] = df['timestamp'].dt.hour
df.head()

,timestamp,equipment_energy_consumption,lighting_energy,zone1_temperature,zone1_humidity,zone2_temperature,zone2_humidity,zone3_temperature,zone3_humidity,zone4_temperature,...,outdoor_humidity,wind_speed,visibility_index,dew_point,random_variable1,random_variable2,month,day,dayofweek,hour
0,2016-01-11 17:10:00,60.0,30.0,19.890000,46.693333,19.2,44.722500,19.79,44.790000,19.000000,...,92.0,6.666667,59.166667,5.200000,18.606195,18.606195,1,11,0,17
1,2016-01-11 17:20:00,50.0,30.0,19.890000,46.300000,19.2,44.626667,19.79,44.933333,35.921144,...,92.0,6.333333,55.333333,5.100000,28.642668,28.642668,1,11,0,17
2,2016-01-11 17:30:00,50.0,40.0,33.746609,46.066667,19.2,44.590000,19.79,45.000000,20.290000,...,92.0,6.000000,51.500000,37.673716,45.410389,45.410389,1,11,0,17
3,2016-01-11 17:40:00,60.0,40.0,19.890000,46.333333,19.2,44.530000,19.79,45.000000,18.890000,...,92.0,5.666667,47.666667,4.900000,10.084097,10.084097,1,11,0,17
4,2016-01-11 17:50:00,50.0,40.0,19.890000,46.026667,19.2,44.500000,19.79,44.933333,18.890000,...,92.0,5.333333,43.833333,4.800000,44.919484,24.780874,1,11,0,17


In [85]:
# Drop the original timestamp column

df.drop(['timestamp'], axis=1, inplace=True)
df.head()

,equipment_energy_consumption,lighting_energy,zone1_temperature,zone1_humidity,zone2_temperature,zone2_humidity,zone3_temperature,zone3_humidity,zone4_temperature,zone4_humidity,...,outdoor_humidity,wind_speed,visibility_index,dew_point,random_variable1,random_variable2,month,day,dayofweek,hour
0,60.0,30.0,19.890000,46.693333,19.2,44.722500,19.79,44.790000,19.000000,45.992500,...,92.0,6.666667,59.166667,5.200000,18.606195,18.606195,1,11,0,17
1,50.0,30.0,19.890000,46.300000,19.2,44.626667,19.79,44.933333,35.921144,45.890000,...,92.0,6.333333,55.333333,5.100000,28.642668,28.642668,1,11,0,17
2,50.0,40.0,33.746609,46.066667,19.2,44.590000,19.79,45.000000,20.290000,45.723333,...,92.0,6.000000,51.500000,37.673716,45.410389,45.410389,1,11,0,17
3,60.0,40.0,19.890000,46.333333,19.2,44.530000,19.79,45.000000,18.890000,45.530000,...,92.0,5.666667,47.666667,4.900000,10.084097,10.084097,1,11,0,17
4,50.0,40.0,19.890000,46.026667,19.2,44.500000,19.79,44.933333,18.890000,45.730000,...,92.0,5.333333,43.833333,4.800000,44.919484,24.780874,1,11,0,17


In [86]:
## create new features based on zone temperature and humidity

zone_temp_cols = []
zone_humid_cols = []

for col in df.columns:
    if "zone" in col and "temperature" in col:
        zone_temp_cols.append(col)
    elif "zone" in col and "humidity" in col:
        zone_humid_cols.append(col)
        
# create new features based on temperature and humidity

df["mean_zone_temperature"] = df[zone_temp_cols].mean(axis=1)
df["mean_zone_humidity"] = df[zone_humid_cols].mean(axis=1)
df["temperature_diff"] = df["outdoor_temperature"] - df["mean_zone_temperature"]
df["humidity_diff"] = df["outdoor_humidity"] - df["mean_zone_humidity"]

df["zone_temperature_std"] = df[zone_temp_cols].std(axis=1)
df["zone_humidity_std"] = df[zone_humid_cols].std(axis=1)
df["zone_temperature_range"] = df[zone_temp_cols].max(axis=1) - df[zone_temp_cols].min(axis=1)

In [87]:
# Weather interactions

df['temp_humidity_interaction'] = df['outdoor_temperature'] * df['outdoor_humidity']

In [88]:
# Lag features (using 1-hour lag)

df['energy_lag_1'] = df['equipment_energy_consumption'].shift(1)

In [89]:
# Rolling statistics

df['rolling_3h_mean'] = df['equipment_energy_consumption'].rolling(window=3).mean()

In [90]:
# Handle missing values

df['energy_lag_1'].fillna(method='bfill', inplace=True)
df['rolling_3h_mean'].fillna(method='bfill', inplace=True)

In [91]:
df.head()

,equipment_energy_consumption,lighting_energy,zone1_temperature,zone1_humidity,zone2_temperature,zone2_humidity,zone3_temperature,zone3_humidity,zone4_temperature,zone4_humidity,...,mean_zone_temperature,mean_zone_humidity,temperature_diff,humidity_diff,zone_temperature_std,zone_humidity_std,zone_temperature_range,temp_humidity_interaction,energy_lag_1,rolling_3h_mean
0,60.0,30.0,19.890000,46.693333,19.2,44.722500,19.79,44.790000,19.000000,45.992500,...,17.149630,50.028302,-10.666296,41.971698,4.025668,13.850293,13.056667,596.466667,60.0,53.333333
1,50.0,30.0,19.890000,46.300000,19.2,44.626667,19.79,44.933333,35.921144,45.890000,...,19.206794,50.628889,-12.840127,41.371111,7.500334,12.757790,29.361144,585.733333,60.0,53.333333
2,50.0,40.0,33.746609,46.066667,19.2,44.590000,19.79,45.000000,20.290000,45.723333,...,18.762216,55.663222,-12.512216,36.336778,6.971894,19.395616,27.313276,575.000000,50.0,53.333333
3,60.0,40.0,19.890000,46.333333,19.2,44.530000,19.79,45.000000,18.890000,45.530000,...,15.679242,50.732963,-9.545909,41.267037,5.916547,13.355827,15.413489,564.266667,50.0,53.333333
4,50.0,40.0,19.890000,46.026667,19.2,44.500000,19.79,44.933333,18.890000,45.730000,...,17.267037,50.255926,-11.250370,41.744074,4.243146,13.978409,13.590000,553.533333,60.0,53.333333
